In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM,Dense,Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
data=pd.read_csv("GOOG.csv")

In [4]:
data=data[['Close']]
data.dropna(inplace=True)

In [5]:
scaler=MinMaxScaler(feature_range=(0,1))
data_scaled=scaler.fit_transform(data)

In [6]:
def create_sequences(data,sequence_length):
    X,Y=[],[]
    for i in range(sequence_length,len(data)):
        X.append(data[i-sequence_length:i,0])
        Y.append(data[i,0])
    return np.array(X),np.array(Y)

In [7]:
sequence_length=60
X,Y=create_sequences(data_scaled,sequence_length)

In [8]:
X=np.reshape(X,(X.shape[0],X.shape[1],1))

In [9]:
split=int(0.8*len(X))
X_train,X_test=X[:split],X[split:]
Y_train,Y_test=Y[:split],Y[split:]

In [10]:
model=keras.Sequential([
    layers.SimpleRNN(50,return_sequences=True,input_shape=(X_train.shape[0],1)),
    layers.SimpleRNN(50),
    layers.Dense(1)
])

C:\Users\Menka khandare\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
history=model.fit(X_train,Y_train,epochs=50,batch_size=32,validation_split=0.1,verbose=1)

Epoch 1/50
81/85 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0331

In [ ]:
loss=model.evaluate(X_test,Y_test)
print(f"Test loss: {loss:.4f}")

In [ ]:
predicted_Stock_price=model.predict(X_test)

In [ ]:
predicted_Stock_price=scaler.inverse_transform(predicted_Stock_price.reshape(-1,1))
y_test_scaled=scaler.inverse_transform(Y_test.reshape(-1,1))

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.plot(y_test_scaled, color='blue', label='Actual Google Stock Price')
plt.plot(predicted_stock_price, color='red', label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()